## Gait Video Study 
### Validating the estimated 3D poses via CoP computed on the treadmill 
### This code qualitatively and quantitatively validates the centroid of base of support of the estimated body coordinates with their respective treadmill extracted CoP's on a frame-wise or support group-wise basis

We will do qualitative and quantitative validation for CoP and centroid of BoS. 
* In this code, we will compile the quantitative results of distances and correlations between CoPs and centroids for each cohort, trial (W/WT) and support type (Single/Double). 
* Further, we will plot the butterfly diagrams for CoPs and centroids to qualitatively vizualize how treadmill CoPs match with BoS's centroid values. 
* We need to select one method of analysis out of shift approach and excursion approach.
* We need to collect statistics of our CoP validation analysis to put in the manuscript. 

In [ ]:
from importlib import reload
from ml_utils import imports 
reload(imports)
from ml_utils.imports import *